In [ ]:
!pip install streamlit 

!npm install localtunnel

!pip install langchain langchain-community 

!pip install sentence-transformers 

!pip install chromadb 

!pip install transformers 

!pip install torch torchvision torchaudio 

!pip install huggingface_hub 

!pip install pypdf 

!pip install bitsandbytes 

!pip install faiss-cpu

^C


In [ ]:
%%writefile app.py
import os
from dotenv import load_dotenv
import streamlit as st
from huggingface_hub import login
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import re
import numpy as np
import torch

import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

login(token=secret_value_0)
generation_args = {
    "max_new_tokens": 256,
    "no_repeat_ngram_size": 4,
    "repetition_penalty": 1.2,
    "temperature": 0.0,
    "do_sample": False,
}

@st.cache_resource
def load_pipeline():
    model_id = 'meta-llama/Llama-3.1-8B-Instruct'
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        low_cpu_mem_usage=True
    )
    return tokenizer, model

@st.cache_resource
def load_faiss():
    input_dir = "/kaggle/input/"
    pdf_data = []
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".pdf"):
                file_path = os.path.join(root, file)
                data_load = PyPDFLoader(file_path)
                data = data_load.load()
                pdf_data.append(data)
    
    if not pdf_data:
        st.error("No PDF files found in the specified directory.")
        return None, None, None

    page_contents = [doc for i in range(len(pdf_data)) for doc in pdf_data[i]]
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
    documents = text_splitter.split_documents(page_contents)

    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v2')
    pdf_embeddings = embedder.encode([doc.page_content for doc in documents])

    dimension = pdf_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    
    index.add(pdf_embeddings)
    
    return index, embedder, documents

# Load the model and FAISS index
tokenizer, model = load_pipeline()
index, embedder, documents = load_faiss()
st.session_state.index = index
st.session_state.embedder = embedder
st.session_state.documents = documents

st.title("Medical ChatBot")

if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

def query_faiss(query):
    query_embedding = st.session_state.embedder.encode([query])
    Dis, Ind = st.session_state.index.search(query_embedding, 10)
    retrieved_docs = [st.session_state.documents[i].page_content for i in Ind[0]]
    return retrieved_docs

def format_chat_template(question, retrieved_context):
    chat_template = f"""
Context:
{retrieved_context}

Instructions:
"You are a helpful assistant that is an expert at extracting the most useful information from the given context above. Also bring in extra relevant infromation to the user query from outside the given context if there any."
"if there any refrences or links in the context just ignore it"
"Answer with the same language as the question"
Question:
{question}

Answer:

"""
    return chat_template

def get_response(query):
    retrieved_docs = query_faiss(query)
    if not retrieved_docs:
        return "Sorry, I couldn't retrieve any relevant documents."
    retrieved_context = " ".join(retrieved_docs)
    formatted_input = format_chat_template(query, retrieved_context)
    inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, **generation_args)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    answer_start = response.find("Answer:") + len("Answer: ")

    only_answer = response[answer_start:].strip()
    
    cleaned_answer = re.sub(r'\s+', ' ', only_answer)
    cleaned_answer = re.sub(r'[^\w\s.,;\[\]\(\)]', '', cleaned_answer)
    sentences = re.findall(r'.*?[.!?]', cleaned_answer)
    if sentences:
        cleaned_answer = ' '.join(sentences)
    
    return cleaned_answer

if prompt := st.chat_input("Ask Anything..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.spinner("Thinking..."):
        response = get_response(prompt)
        
    with st.chat_message("assistant"):
        st.markdown(response)

    st.session_state.messages.append({"role": "assistant", "content": response})


In [ ]:
!streamlit run /kaggle/working/app.py  & npx localtunnel --port 8501